In [1]:
import logging
import pickle
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from keras.models import Model, Sequential
from keras.layers import *
from keras.preprocessing.text import one_hot
from keras.optimizers import Adam
from tqdm import tnrange, tqdm_notebook
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.utils import class_weight
from keras.layers import Concatenate, Dense, LSTM, Input, Activation, concatenate

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
set_session(tf.Session(config=config))

In [3]:
MODEL_PATH = "../models/"
DATA_PATH = "../data/"

In [4]:
train = pd.read_csv(DATA_PATH + "preprocessed/train_ling.csv")

In [5]:
targets = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

In [6]:
meta_features = ['count_sent', 'count_word', 'count_unique_word', 'count_letters',
       'count_punctuations', 'count_words_upper', 'count_words_title',
       'count_stopwords', 'mean_word_len', 'word_unique_percent',
       'punct_percent', 'count_swear_words']

In [7]:
train.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,count_sent,...,count_unique_word,count_letters,count_punctuations,count_words_upper,count_words_title,count_stopwords,mean_word_len,word_unique_percent,punct_percent,count_swear_words
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,2,...,41,264,10,2,11,16,5.162791,95.348837,23.255814,0
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1,...,17,112,12,1,3,2,5.588235,100.000000,70.588235,0
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1,...,39,233,6,0,2,19,4.571429,92.857143,14.285714,0
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,5,...,82,622,21,5,7,55,4.486726,72.566372,18.584071,0
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1,...,13,67,5,0,2,5,4.230769,100.000000,38.461538,0


In [8]:
def build_input_data(sentences, labels, vocabulary):
    x = np.array([[embeddings_index[vocabulary_inv[vocabulary['word']]] if word in vocabulary.keys() else len(vocabulary) - 1 for word in sentence] for sentence in sentences])
    y = np.array(labels)
    return [x, y]

In [9]:
train_text = train['comment_text']

In [10]:
tokenizer = Tokenizer(num_words=18400)
tokenizer.fit_on_texts(train_text)
sequences = tokenizer.texts_to_sequences(train_text)

In [11]:
vocabulary = tokenizer.word_index
vocabulary_inv = {v:k for k, v in vocabulary.items()}
embeddings_index = {}
EMBEDDING_DIM = 100
f = open("../../../embeddings/glove.6B." + str(EMBEDDING_DIM) + "d.txt")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [12]:
print('Found %s word vectors.' % len(embeddings_index))
embedding_matrix = np.zeros((len(vocabulary) + 1, EMBEDDING_DIM))
embedding_matrix[-1] = np.random.rand(EMBEDDING_DIM) # oov-vector
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i - 1] = embedding_vector
embedding_layer = Embedding(embedding_matrix.shape[0],
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=200, trainable = False)

Found 400000 word vectors.


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_train, x_dev, y_train, y_dev = train_test_split(train, targets, test_size=0.3, random_state=42)

In [15]:
max_length = 200

x_train_texts = tokenizer.texts_to_sequences(x_train['comment_text'])
x_train_texts = pad_sequences(x_train_texts, maxlen=max_length, padding='post')

x_dev_texts = tokenizer.texts_to_sequences(x_dev['comment_text'])
x_dev_texts = pad_sequences(x_dev_texts, maxlen=max_length, padding='post')

In [16]:
sequence_input = Input(shape=(max_length, ))
embedded_sequences = embedding_layer(sequence_input)
x = SpatialDropout1D(0.2)(embedded_sequences)
x = Bidirectional(GRU(80, return_sequences=True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = concatenate([avg_pool, max_pool])
outp = Dense(6, activation="sigmoid")(conc)
model = Model(inputs=sequence_input, outputs=outp)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 100)     21033800    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 200, 100)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 200, 160)     86880       spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
global_ave

In [17]:
from keras.callbacks import ModelCheckpoint

In [ ]:
filepath="../models/pooled_gru-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.fit(x_train_texts, y_train, validation_data=(x_dev_texts, y_dev),
          epochs=20, batch_size=20, callbacks=callbacks_list)

Train on 111699 samples, validate on 47872 samples
Epoch 1/20
111699/111699 [==============================] - 3102s 28ms/step - loss: 0.0800 - acc: 0.9750 - val_loss: 0.0700 - val_acc: 0.9771

Epoch 00001: val_loss improved from -inf to 0.07003, saving model to ../models/pooled_gru-01-0.07.hdf5
Epoch 2/20
111699/111699 [==============================] - 3084s 28ms/step - loss: 0.0682 - acc: 0.9779 - val_loss: 0.0636 - val_acc: 0.9790

Epoch 00002: val_loss did not improve
Epoch 3/20
111699/111699 [==============================] - 3622s 32ms/step - loss: 0.0637 - acc: 0.9788 - val_loss: 0.0673 - val_acc: 0.9789

Epoch 00003: val_loss did not improve
Epoch 4/20
111699/111699 [==============================] - 3429s 31ms/step - loss: 0.0602 - acc: 0.9798 - val_loss: 0.0603 - val_acc: 0.9798

Epoch 00004: val_loss did not improve
Epoch 5/20
111699/111699 [==============================] - 4973s 45ms/step - loss: 0.0582 - acc: 0.9803 - val_loss: 0.0582 - val_acc: 0.9805

Epoch 00005: val_

In [24]:
model.evaluate(x_dev_texts, y_dev)

47872/47872 [==============================] - 162s 3ms/step


[0.0589507606406858, 0.9809004756457665]

In [25]:
model_json = model.to_json()
with open(MODEL_PATH + "keras_pooled_gru_classification_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(MODEL_PATH + "keras_pooled_gru_classification_model.h5")
print("Saved model to disk")

Saved model to disk


In [26]:
model_name = 'keras_pooled_gru_classification_model'
X_official_test = pd.read_csv("../data/raw/test.csv")
x_dev_texts = tokenizer.texts_to_sequences(X_official_test['comment_text'])
x_dev_texts = pad_sequences(x_dev_texts, maxlen=max_length, padding='post')
pred = model.predict(x_dev_texts)
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
sample_submission = pd.read_csv('../submissions/sample_submission.csv')
sample_submission[list_classes] = pred
sample_submission.to_csv("../submissions/" + model_name + ".csv", index=False)